In [2]:
# Load returns
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.covariance import LedoitWolf
from sklearn.decomposition import PCA

PROC_DIR = Path("../data/processed").resolve()

def load_log_returns(path=PROC_DIR / "log_returns.parquet") -> pd.DataFrame:
    df = pd.read_parquet(path)
    df["date"] = pd.to_datetime(df["date"])
    df = df.set_index("date").sort_index()
    return df

R = load_log_returns()
R.shape, R.tail()


((5277, 41),
                 AAPL      AMZN       BAC     BRK-B      COST       CVX  \
 date                                                                     
 2025-12-17 -0.010138 -0.005813 -0.004755  0.008824  0.002623  0.018700   
 2025-12-18  0.001287  0.024508 -0.005330 -0.001747 -0.005883 -0.012315   
 2025-12-19  0.005423  0.002599  0.018443 -0.017757 -0.002300  0.000406   
 2025-12-22 -0.009915  0.004739  0.010976  0.010900 -0.006590  0.013779   
 2025-12-23  0.003812  0.000000  0.002770  0.001299  0.003224  0.004909   
 
                  DIA       EEM       EFA       GLD  ...       XLB       XLE  \
 date                                                ...                       
 2025-12-17 -0.004533 -0.007198 -0.008145  0.008552  ...  0.004213  0.021899   
 2025-12-18  0.001479  0.011342  0.007091 -0.001805  ... -0.000443 -0.014622   
 2025-12-19  0.003506  0.009727  0.006727  0.001128  ...  0.004418  0.000000   
 2025-12-22  0.004790  0.005384  0.002511  0.022819  ...  0.

In [3]:
# Define Regiimes (normal vs stressed)
def pick_windows(
    R: pd.DataFrame,
    normal_days: int = 252,
    stress_days: int = 252,
    lookback_days: int = 2520,  # ~10y
):
    Rlb = R.tail(lookback_days).dropna(axis=1, how="any")  # strict panel
    if len(Rlb) < max(normal_days, stress_days) + 5:
        raise ValueError("Not enough data for requested windows.")

    # Normal = most recent window
    normal = Rlb.tail(normal_days)

    # Stress = rolling window with max realized vol (equal-weight proxy)
    ew = Rlb.mean(axis=1)
    roll_vol = ew.rolling(stress_days).std() * np.sqrt(252)
    end_idx = roll_vol.idxmax()
    end_loc = Rlb.index.get_loc(end_idx)
    stress = Rlb.iloc[end_loc - stress_days + 1 : end_loc + 1]

    return normal, stress

R_normal, R_stress = pick_windows(R)
R_normal.shape, R_stress.shape, (R_normal.index.min(), R_normal.index.max()), (R_stress.index.min(), R_stress.index.max())


((252, 41),
 (252, 41),
 (Timestamp('2024-12-20 00:00:00'), Timestamp('2025-12-23 00:00:00')),
 (Timestamp('2020-01-31 00:00:00'), Timestamp('2021-01-29 00:00:00')))

In [4]:
# Core Math
def portfolio_vol(cov: np.ndarray, w: np.ndarray, ann_factor: float = 252.0) -> float:
    var = float(w.T @ cov @ w)
    return np.sqrt(var * ann_factor)

def mcr_and_rc(cov: np.ndarray, w: np.ndarray):
    # MCR_i = (Σ w)_i / σ_p  (using daily σ_p)
    port_var = float(w.T @ cov @ w)
    if port_var <= 0:
        raise ValueError("Non-positive portfolio variance.")
    port_sigma = np.sqrt(port_var)

    marginal = (cov @ w) / port_sigma          # vector
    risk_contrib = w * marginal                # vector, sums to σ_p (daily)
    risk_share = risk_contrib / risk_contrib.sum()
    return port_sigma, marginal, risk_contrib, risk_share

def corr_from_cov(cov: np.ndarray) -> np.ndarray:
    d = np.sqrt(np.diag(cov))
    denom = np.outer(d, d)
    with np.errstate(divide="ignore", invalid="ignore"):
        corr = cov / denom
    corr[np.isnan(corr)] = 0.0
    corr[np.isinf(corr)] = 0.0
    np.fill_diagonal(corr, 1.0)
    return corr


In [5]:
# PCA outputs (eigenvalues, explained variance, effective bets)
def pca_from_cov(cov: np.ndarray):
    # PCA on covariance: eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(cov)  # ascending
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    total = eigvals.sum()
    exp_var = eigvals / total if total > 0 else np.zeros_like(eigvals)

    # Effective number of bets: 1 / sum(p^2), where p = explained variance shares
    enb = 1.0 / float(np.sum(exp_var**2)) if total > 0 else 0.0
    return eigvals, exp_var, eigvecs, enb


In [6]:
# Function to return regime payload
def compute_regime_metrics(Rwin: pd.DataFrame, weights: pd.Series):
    # Align universe
    cols = [c for c in weights.index if c in Rwin.columns]
    Rw = Rwin[cols].dropna()
    w = weights[cols].astype(float).values
    w = w / w.sum()

    # Sample covariance (daily)
    cov = np.cov(Rw.values, rowvar=False, ddof=1)
    corr = corr_from_cov(cov)

    # Shrinkage covariance (Ledoit-Wolf)
    lw = LedoitWolf().fit(Rw.values)
    cov_lw = lw.covariance_
    corr_lw = corr_from_cov(cov_lw)

    # Risk
    sigma_p, mcr, rc, rshare = mcr_and_rc(cov_lw, w)
    vol_ann = sigma_p * np.sqrt(252)

    # PCA
    eigvals, exp_var, eigvecs, enb = pca_from_cov(cov_lw)

    out = {
        "universe": cols,
        "n_obs": int(len(Rw)),
        "cov_lw": cov_lw,
        "corr_lw": corr_lw,
        "vol_ann": float(vol_ann),
        "weights": w,
        "mcr": mcr,
        "risk_contrib": rc,
        "risk_share": rshare,
        "eigvals": eigvals,
        "explained_var": exp_var,
        "loadings": eigvecs,  # columns are PCs
        "effective_bets": float(enb),
    }
    return out


In [7]:
# Define Portfolio weights (temp equal, later user input)
# For now: equal-weight across available columns
universe = sorted(set(R_normal.columns).intersection(R_stress.columns))
weights = pd.Series(1.0, index=universe)

normal_metrics = compute_regime_metrics(R_normal, weights)
stress_metrics = compute_regime_metrics(R_stress, weights)

normal_metrics["vol_ann"], stress_metrics["vol_ann"], normal_metrics["effective_bets"], stress_metrics["effective_bets"]


(0.12347144697161923,
 0.28701511804334856,
 5.488884260124275,
 2.1602271701203595)

In [8]:
# Store the payload for the web UI
OUT_DIR = Path("../data/processed/risk_payloads").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

def save_payload(name: str, m: dict):
    cols = m["universe"]
    df_assets = pd.DataFrame({
        "symbol": cols,
        "weight": m["weights"],
        "mcr": m["mcr"],
        "risk_contrib": m["risk_contrib"],
        "risk_share": m["risk_share"],
    })
    df_assets.to_parquet(OUT_DIR / f"{name}_asset_metrics.parquet", index=False)

    pd.DataFrame(m["corr_lw"], index=cols, columns=cols).reset_index(names="symbol").to_parquet(
        OUT_DIR / f"{name}_corr.parquet", index=False
    )
    pd.DataFrame(m["cov_lw"], index=cols, columns=cols).reset_index(names="symbol").to_parquet(
        OUT_DIR / f"{name}_cov.parquet", index=False
    )

    df_pca = pd.DataFrame({
        "pc": np.arange(1, len(m["eigvals"]) + 1),
        "eigval": m["eigvals"],
        "explained_var": m["explained_var"],
    })
    df_pca.to_parquet(OUT_DIR / f"{name}_pca.parquet", index=False)

save_payload("normal", normal_metrics)
save_payload("stress", stress_metrics)

print("Saved to", OUT_DIR)


Saved to C:\Users\matt5\Documents\code\portfolio-risk\data\processed\risk_payloads
